In [1]:
import json
import os
import numpy as np
import pandas as pd
from datasets import load_dataset
from urllib.parse import parse_qs, urlparse
import requests
import chromadb
import openai
from chromadb.config import Settings
import os
import openai
from openai import OpenAI
import chromadb
import fitz  # PyMuPDF
from chromadb.utils import embedding_functions
import chromadb.utils.embedding_functions as embedding_functions
from chromadb.config import Settings
import base64

c:\Users\coolk\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import base64


def extract_pdf_url(url):
    """
    Extracts the actual PDF URL from the given URL.
    Decodes it from base64 if necessary.
    """
    if url.lower().endswith('.pdf'):
        return url  # Direct PDF URL
    else:
        parsed_url = urlparse(url)
        query_params = parse_qs(parsed_url.query)
        pdf_target = query_params.get('pdfTarget', [None])[0]

        if pdf_target:
            pdf_url = base64.b64decode(pdf_target).decode('utf-8')
            return pdf_url
        else:
            raise ValueError("No valid PDF URL found in the provided URL")

def download_pdf(url, save_path):
    """
    Downloads a PDF from a given URL.
    """
    try:
        pdf_url = extract_pdf_url(url)
        response = requests.get(pdf_url, stream=True)
        response.raise_for_status()  # Ensure the request was successful

        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        print(f"Downloaded PDF from: {pdf_url} to {save_path}")
    except Exception as e:
        print(f"Error downloading PDF: {e}")
        
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

In [4]:
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder(
    "cross-encoder/ms-marco-TinyBERT-L-2-v2", max_length=512, device="cpu"
)

c:\Users\coolk\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\coolk\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\coolk\.cache\huggingface\hub\models--cross-encoder--ms-marco-TinyBERT-L-2-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either n

In [5]:
reranked_docs = cross_encoder.rank(
    question,
    [doc.page_content for doc in retrieved_docs],
    top_k=3,
    return_documents=True,
)

NameError: name 'question' is not defined

In [6]:
def create_chroma_vectordb_from_pdf(pdf_path, openai_api_key, batch_size=100):
# Extract text from PDF
    text = extract_text_from_pdf(pdf_path)
    
    # Split text into sentences
    sentences = text.split('\n')
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]  # Remove empty sentences
    
    # Initialize OpenAI Embedding Function
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(
        api_key=openai_api_key,
        model_name="text-embedding-ada-002"
    )
    
    # Batch processing for embeddings
    vectors = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i+batch_size]
        if len(batch) > 0:  # Ensure batch is not empty
            batch_vectors = openai_ef(batch)
            vectors.extend(batch_vectors)
    
    # Store vectors in Chroma vector database
    client = chromadb.Client(Settings())
    collection_name = "Finance_bench_documents"
    collection = client.get_or_create_collection(name= collection_name) 
    # if client.has_collection(collection_name):
    #     collection = client.get_collection(collection_name)
    # else:
    #     collection = client.create_collection(collection_name)
    
    for i, (sentence, vector) in enumerate(zip(sentences, vectors)):
        collection.add(f"id_{i}", vector, {"sentence": sentence})
    
    print(f"Stored {len(sentences)} vectors in the Chroma vector database.")

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(sec_filing_pdf)

# Load the PDF document
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L12-v2"),
    chunk_size=256,
    chunk_overlap=16,
    strip_whitespace=True,
)

docs = text_splitter.split_documents(pages)

text_splitter = CharacterTextSplitter(
            chunk_size=1300, 
            chunk_overlap=5,
            #separators=["\n\n", "\n", " ", ""],
            length_function=len)
docs = text_splitter.split_documents(documents)

NameError: name 'sec_filing_pdf' is not defined